In [1]:
import zerorpc
from gensim.models import Word2Vec
from gensim.models import KeyedVectors 
list_csv = ['data/attraction_places.csv',
           'data/hotel_places.csv',
           'data/restaurant_places.csv']
attraction_tag  = KeyedVectors.load('model/attraction_tag_e5.model')
attraction_user = KeyedVectors.load('model/attraction_user.model')
hotel_tag       = KeyedVectors.load('model/hotel_tag_e5.model')
hotel_user      = KeyedVectors.load('model/hotel_user.model')
restaurant_tag  = KeyedVectors.load('model/restaurant_tag_e5.model')
restaurant_user = KeyedVectors.load('model/restaurant_user.model')
class Recomm:
    def test(self):
        return 'test string'
    def rank(srcwv, entry, positive=None, negative=None):
        subwv = srcwv[entry]
        newwv = Word2Vec(size=len(subwv[0]))
        # add entries to new vector
        for i in range(len(entry)):
            newwv.wv.add(entry[i], subwv[i])
        # add p, n to new vector
        for w in positive:
            newwv.wv.add(w, srcwv[w])
        for w in negative:
            newwv.wv.add(w, srcwv[w])
        ranked = newwv.most_similar(positive=positive,
                                  negative=negative,
                                  topn=len(entry))
        return ranked
    def stop(self):
        raise Exception

C:\Users\CypressRH\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
import time
try:
    server = zerorpc.Server(Recomm())
    server.bind("tcp://0.0.0.0:4242")
    server.run()
    time.sleep(3)
    server.stop()
except Exception as e:
    print('exception caught')